In [143]:
using LinearAlgebra, Distributions, Statistics, SparseArrays

In [162]:
include("../src/cmvn.jl")
include("../src/generate.jl")
include("../src/mvn.jl")

expt_tnorm

In [163]:
iters = 10
ms = [16, 32]; ds = [1, 2, 4, 8, 16]
ans_1 = zeros(length(ms), length(ds)); time_1 = zeros(length(ms), length(ds))
# ans_2 = zeros(length(ms), length(ds)); time_2 = zeros(length(ms), length(ds))

for i in 1:length(ms)
    for j in 1:length(ds)
        m = ms[i]; d = ds[j]
        for p in 1:iters
            Σ = Σ_generate(m)
            a = fill(-Inf, m)
            b = rand(Uniform(0, m), m)
            if d ≤ 4
                time_1[i, j] += @elapsed (ans_1[i, j] += CMVN(Σ, a, b, d, ns = 5, N = 200)[1])
#             time_2[i, j] += @elapsed ans_2[i, j] += RCMVN(Σ, a, b, d, m)[1]
            elseif d ≤ 8
                time_1[i, j] += @elapsed (ans_1[i, j] += CMVN(Σ, a, b, d, ns = 10, N = 400)[1])
#             time_2[i, j] += @elapsed ans_2[i, j] += RCMVN(Σ, a, b, d, m)[1]
            else
                time_1[i, j] += @elapsed (ans_1[i, j] += CMVN(Σ, a, b, d, ns = 10, N = 1000)[1])
#             time_2[i, j] += @elapsed ans_2[i, j] += RCMVN(Σ, a, b, d, m)[1]
            end
        end
    end
end
ans_1 ./= iters; time_1 ./= iters
# ans_2 ./= iters; time_2 ./= iters

UndefVarError: UndefVarError: c not defined

In [161]:
# todo: fix NaN issue
ans_1

2×5 Array{Float64,2}:
 0.543199  0.772429  NaN  NaN    0.808352
 0.670512  0.590654  NaN  NaN  NaN       

In [95]:
time_1

2×5 Array{Float64,2}:
 6.59825e-5   0.00532465  0.0127773  0.0921083  0.555371
 0.000150362  0.00869017  0.0221256  0.190253   1.16388 

In [168]:
using Random 
Random.seed!(10);

In [169]:
m = 32; d = 16
Σ = Σ_generate(m)
a = fill(-Inf, m)
b = rand(Uniform(0, m), m)
CMVN(Σ, a, b, d, ns = 10, N = 1000)

UndefVarError: UndefVarError: c not defined

In [170]:
ns = 10; N = 1000; tol = 1e-8
s = Int(m / d)
y = zeros(m)
P = 1.
L, D = LDL(copy(Σ), d)

([1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.005570006607063755 -0.12384007628050012 … 1.0 0.0; -0.09415837276017197 0.03215257017896041 … 0.0 1.0], Array{Float64,2}[[0.4328229505912568 0.047261477120845635 … 0.016510750484024627 -0.0012522933165029677; 0.047261477120845635 0.41040008719761584 … -0.07885749679851234 -0.005775578570524403; … ; 0.016510750484024627 -0.07885749679851234 … 0.43615034043576695 -0.05914581271132365; -0.0012522933165029677 -0.005775578570524403 … -0.05914581271132365 0.5267829499639183], [0.28110274515960065 -0.08039868827683813 … -0.018772671981476637 0.0567370181230928; -0.08039868827683813 0.38720248385327727 … 0.02910609804697727 -0.07605819832148084; … ; -0.01877267198147664 0.029106098046977273 … 0.18286661027042594 -0.09904182944590384; 0.0567370181230928 -0.07605819832148084 … -0.09904182944590384 0.35149536925312025]])

In [171]:
i = 1
j = Int((i - 1) * d)
g = L[(j + 1):(j + d), 1:j] * y[1:j]
a1 = a[(j + 1):(j + d)] .- g
b1 = b[(j + 1):(j + d)] .- g
D1 = Symmetric(D[i])
L1 = cholesky(D1).L
# P *= cdf_trunnormal(a1, b1, zeros(d), copy(Symmetric(D1)))
# y[(j + 1):(j + d)] .= ex_trunnormal(a1, b1, zeros(d), copy(Symmetric(D1)))
# use ns=10 and N=1000 which is same in Genton2018
p_i = mvn(L1, a1, b1, ns = ns, N = N, tol = tol)
y_i = expt_tnorm(a1, b1, L1, ns = ns, N = N, tol = tol)

P *= p_i
y[(j + 1):(j + d)] .= y_i

UndefVarError: UndefVarError: c not defined

In [172]:
expt_tnorm(a1, b1, L1, ns = ns, N = N, tol = tol)

UndefVarError: UndefVarError: c not defined

In [173]:
c = zeros(d)
Σ1 = L1 * transpose(L1)
μ = zeros(d)

16-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [174]:
for l in 1:d
    μ1 = copy(μ[1:d .!= l] + Σ1[1:d .!= l, l] * (a1[l] - μ[l]) / Σ1[l, l])
    μ2 = copy(μ[1:d .!= l] + Σ1[1:d .!= l, l] * (b1[l] - μ[l]) / Σ1[l, l])
    Σl = copy(Symmetric(Σ1[1:d .!= l, 1:d .!= l] - Σ1[l, 1:d .!= l] * transpose(Σ1[1:d .!= l, l]) / Σ1[l, l]))
    Ll = cholesky(Σl).L
    c[l] = pdf(Normal(μ[l], sqrt(Σ1[l, l])), a1[l]) * 
        mvn(Ll, a1[1:d .!= l], b1[1:d .!= l], ns = ns, N = N, tol = tol, μ = μ1) - 
        pdf(Normal(0, sqrt(Σ1[l, l])), b1[l]) * 
        mvn(Ll, a1[1:d .!= l], b1[1:d .!= l], ns = ns, N = N, tol = tol, μ = μ2)
end

UndefVarError: UndefVarError: c not defined

In [175]:
l  = 1
μ1 = copy(μ[1:d .!= l] + Σ1[1:d .!= l, l] * (a1[l] - μ[l]) / Σ1[l, l])
μ2 = copy(μ[1:d .!= l] + Σ1[1:d .!= l, l] * (b1[l] - μ[l]) / Σ1[l, l])
Σl = copy(Symmetric(Σ1[1:d .!= l, 1:d .!= l] - Σ1[l, 1:d .!= l] * transpose(Σ1[1:d .!= l, l]) / Σ1[l, l]))
Ll = cholesky(Σl).L

15×15 LowerTriangular{Float64,Array{Float64,2}}:
  0.636584      ⋅             ⋅          …    ⋅           ⋅          ⋅      
  0.0770167    0.619942       ⋅               ⋅           ⋅          ⋅      
 -0.0504319   -0.00144694    0.60536          ⋅           ⋅          ⋅      
  0.0781843    0.0299597    -0.0366933        ⋅           ⋅          ⋅      
  0.0977443   -0.143505      0.043785         ⋅           ⋅          ⋅      
 -0.12587     -0.0493789    -0.0916771   …    ⋅           ⋅          ⋅      
 -0.0591913   -0.123959      0.0810631        ⋅           ⋅          ⋅      
 -0.00827497  -0.0917928     0.0306592        ⋅           ⋅          ⋅      
  0.0620673   -0.000413403   0.00649418       ⋅           ⋅          ⋅      
  0.0499591    0.0501746     0.100717         ⋅           ⋅          ⋅      
 -0.138098    -0.0448619     0.00074833  …    ⋅           ⋅          ⋅      
 -0.0180927    0.144444     -0.0275294        ⋅           ⋅          ⋅      
 -0.0646662   -0.0373508   

In [176]:
mvn(Ll, a1[1:d .!= l], b1[1:d .!= l], ns = ns, N = N, tol = tol, μ = μ1)

UndefVarError: UndefVarError: c not defined

In [177]:
n = size(Ll, 1)

15

In [178]:
a2 = a1[1:d .!= l]
b2 = b1[1:d .!= l]

15-element Array{Float64,1}:
 15.340254258608638 
 16.421221478933674 
  8.405633129917412 
 17.3789923921695   
  6.89202288716362  
 29.841535182374095 
  2.8182280368039088
 18.687672604452068 
  7.53400728708678  
 11.293737471216843 
 22.649429446930675 
  9.337462313877921 
  3.3104907442265414
 31.31944283963685  
  8.487006409689812 

In [179]:
for i in 1:n
    if a2[i] != -Inf
        a2[i] = a2[i] - μ1[i]
    end

    if b2[i] != Inf
        b2[i] = b2[i] - μ1[i]
    end
end

In [180]:
T = Float64
values = Vector{T}(undef, ns) 
X = Matrix{T}(undef, n, N)
ax = reshape(repeat(a2, N), n, N)
bx = reshape(repeat(b2, N), n, N)

15×1000 Array{Float64,2}:
  Inf   Inf   Inf   Inf   Inf   Inf  …   Inf   Inf   Inf   Inf   Inf   Inf
  Inf   Inf   Inf   Inf   Inf   Inf      Inf   Inf   Inf   Inf   Inf   Inf
  Inf   Inf   Inf   Inf   Inf   Inf      Inf   Inf   Inf   Inf   Inf   Inf
 -Inf  -Inf  -Inf  -Inf  -Inf  -Inf     -Inf  -Inf  -Inf  -Inf  -Inf  -Inf
 -Inf  -Inf  -Inf  -Inf  -Inf  -Inf     -Inf  -Inf  -Inf  -Inf  -Inf  -Inf
 -Inf  -Inf  -Inf  -Inf  -Inf  -Inf  …  -Inf  -Inf  -Inf  -Inf  -Inf  -Inf
  Inf   Inf   Inf   Inf   Inf   Inf      Inf   Inf   Inf   Inf   Inf   Inf
 -Inf  -Inf  -Inf  -Inf  -Inf  -Inf     -Inf  -Inf  -Inf  -Inf  -Inf  -Inf
  Inf   Inf   Inf   Inf   Inf   Inf      Inf   Inf   Inf   Inf   Inf   Inf
 -Inf  -Inf  -Inf  -Inf  -Inf  -Inf     -Inf  -Inf  -Inf  -Inf  -Inf  -Inf
  Inf   Inf   Inf   Inf   Inf   Inf  …   Inf   Inf   Inf   Inf   Inf   Inf
  Inf   Inf   Inf   Inf   Inf   Inf      Inf   Inf   Inf   Inf   Inf   Inf
  Inf   Inf   Inf   Inf   Inf   Inf      Inf   Inf   Inf   Inf   Inf   Inf

In [181]:
using Primes
if n == 2
    prime_n = [2, 3]
else
    prime_n = Primes.primes(Int(floor(5*n*log(n+1)/4)))
end

15-element Array{Int64,1}:
  2
  3
  5
  7
 11
 13
 17
 19
 23
 29
 31
 37
 41
 43
 47

In [182]:
q = Vector{T}(undef, n)
for i in 1:n
    q[i] = sqrt(prime_n[i])
end

In [183]:
q

15-element Array{Float64,1}:
 1.4142135623730951
 1.7320508075688772
 2.23606797749979  
 2.6457513110645907
 3.3166247903554   
 3.605551275463989 
 4.123105625617661 
 4.358898943540674 
 4.795831523312719 
 5.385164807134504 
 5.5677643628300215
 6.082762530298219 
 6.4031242374328485
 6.557438524302    
 6.855654600401044 

In [184]:
xr = rand(T, n, 1) # xr ~ U(0,1)
for j in 1:N
    X[:,j] = q * (1+j) + xr
end
X = map(x->abs(2*(x-floor(x))-1), X)
p = mvndns(n, N, Ll, X, ax, bx, tol)

UndefVarError: UndefVarError: c not defined

In [187]:
mvndns

mvndns (generic function with 1 method)

In [188]:
mvndns(n, N, Ll, X, ax, bx, tol)

UndefVarError: UndefVarError: c not defined

`mvndns()` 함수에서 문제가 발생하고 있는 것으로 보임

`c` not defined error?